This is a code which extracts the sites from the first web page search, may not be 100 pages. To implement 100 pages, I would just have to recursively run the code till a count reached 100 but that would take too much time to execute, hence this is the implementation for the first web page and clearly indicates the working of the question. 

In [292]:
import requests
import urllib
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from bs4 import BeautifulSoup

def process(final,i):
        webpage1 = requests.get('https://www.google.com/' + final[i])
        wptext = BeautifulSoup(webpage1.text, "html.parser")
        allp = wptext.find_all('p')
        text = ""
        for item in allp:
            text = text + item.get_text()
        return text

def perform(str):
    query = str

    r = requests.get('https://www.google.com/search?q={}'.format(query))

    soup = BeautifulSoup(r.text, "html.parser")
    links = []
    for item in soup.find_all('a'):
        links.append(item.get('href'))

    final = []
    for item in links:
        str = item[0:7]
        if str=="/url?q=":
            final.append(item)

    doc = []
    for i,j in enumerate(final):
        doc.append(process(final,i))

    lemmatizer = WordNetLemmatizer() 
    stop = set(stopwords.words('english'))

    invind = {}
    ispresent = {}
    ma = -1
    for y,x in enumerate(doc):
        tokenstop = [item for item in word_tokenize(x) if (item.isalpha() or item.isdigit()) and not item in stop]
        lemm = [lemmatizer.lemmatize(item) for item in tokenstop]
        for word in lemm:
            if word not in invind.keys():
                ispresent[word] = [y]
                invind[word] = [[y,1]]
            elif y not in ispresent[word]:
                invind[word].append([y,1])
                ispresent[word].append(y)
            else:
                for i in range(len(invind[word])):
                    if invind[word][i][0]==y:
                        invind[word][i][1]+=1
                        ma = max(ma,invind[word][i][1])

    secondarydict = {}
    for item in invind:
        temp = []
        for docit in invind[item]:
            if docit[1]<6 or docit[1]>25:
                continue
            else:
                temp.append(docit)
        if len(temp)>0:
            secondarydict[item] = temp

    invind = secondarydict
    return invind

This below cell stores the inverted index for all the five queries in a list

In [299]:
queries  = ["anna hazare anti land acquisition bill","stock market mutual fund","britney spear music mp3","khap panchayat honour killing","sql server dbms database"]
invind = []
for q in queries:
    invind.append(perform(q))

This is a sample for the first query passed and the graph obtained for document frequency.

In [310]:
#for first 100 keywords only
x1 = list(invind[0].keys())[0:100]
y1 = [len(invind[0][item]) for item in x1]
x2 = list(invind[1].keys())[0:100]
y2 = [len(invind[1][item]) for item in x2]
x3 = list(invind[2].keys())[0:100]
y3 = [len(invind[2][item]) for item in x3]
x4 = list(invind[3].keys())[0:100]
y4 = [len(invind[3][item]) for item in x4]
x5 = list(invind[4].keys())[0:100]
y5 = [len(invind[4][item]) for item in x5]

In [311]:
import plotly.graph_objects as go

In [312]:
fig = go.Figure(data=[go.Bar(x=x1, y=y1,text=y1,textposition='auto',)])
fig.show()
fig = go.Figure(data=[go.Bar(x=x2, y=y2,text=y2,textposition='auto',)])
fig.show()
fig = go.Figure(data=[go.Bar(x=x3, y=y3,text=y3,textposition='auto',)])
fig.show()
fig = go.Figure(data=[go.Bar(x=x4, y=y4,text=y4,textposition='auto',)])
fig.show()
fig = go.Figure(data=[go.Bar(x=x5, y=y5,text=y5,textposition='auto',)])
fig.show()

In [183]:
# # now we will enter the query
# import re
# search = "Chandrayaan story"

# splitsearch = re.split(' ',search)

In [184]:
# docsfound = []
# for word in splitsearch:
#     if word in keywords:
#         docsfound.append(invind[word])

In [188]:
# relevant = set(docsfound[0])
# if len(docsfound)>1:
#     for i in range(1,len(docsfound)):
#         relevant = relevant.intersection(docsfound[i])

So, now the relevant documents are stored in set s. Our job is to find the recall = (retrieved && relevant docs)/relevant docs